# Quickstart

Build applications quickly using **preset agents**. First example - build agent that uses tools.

Below we use open-ai model. To use any other, simple change *model_client*. Check more at [models](https://microsoft.github.io/autogen/stable//user-guide/agentchat-user-guide/tutorial/models.html)

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [34]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Define a model client. You can use other model client that implements
# the `ChatCompletionClient` interface.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    # api_key="YOUR_API_KEY",
)


# Define a simple function tool that the agent can use.
# For this example, we use a fake weather tool for demonstration purposes.
async def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    return f"The weather in {city} is 73 degrees and Sunny."


# Define an AssistantAgent with the model, tool, system message, and reflection enabled.
# The system message instructs the agent via natural language.
agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[get_weather],
    system_message="You are a helpful assistant.",
    reflect_on_tool_use=True,
    model_client_stream=True,  # Enable streaming tokens from the model client.
)


# Run the agent and stream the messages to the console.
async def main() -> None:
    await Console(agent.run_stream(task="What is the weather in New York?"))
    # Close the connection to the model client.
    await model_client.close()


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

---------- TextMessage (user) ----------
What is the weather in New York?
---------- ToolCallRequestEvent (weather_agent) ----------
[FunctionCall(id='call_KGqmBu6EPZf3taOazrsAMoOQ', arguments='{"city":"New York"}', name='get_weather')]
---------- ToolCallExecutionEvent (weather_agent) ----------
[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', name='get_weather', call_id='call_KGqmBu6EPZf3taOazrsAMoOQ', is_error=False)]
---------- ModelClientStreamingChunkEvent (weather_agent) ----------
The weather in New York is currently 73 degrees and sunny.


### Playing around with above code

In [35]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

model = OpenAIChatCompletionClient(model='gpt-4o-mini')

# function that calculates factorial
async def factorial(n : int) -> int:
    """ This function returns the factorial of an input integer """
    if n < 2:
        return 1
    else:
        return n * await factorial(n-1)


agent = AssistantAgent(
    name="factorial_creator",
    model_client=model,
    tools=[factorial],
    system_message="You are a math wizard. Specifically, a factorial creator.",
    reflect_on_tool_use=True,
    model_client_stream=True
)

async def main() -> None:
    await Console(agent.run_stream(task="What is the factorial of 5?"))
    await model.close()

await main()

---------- TextMessage (user) ----------
What is the factorial of 5?
---------- ToolCallRequestEvent (factorial_creator) ----------
[FunctionCall(id='call_wpLjDwnZ3zRQFu1Kaw35msnN', arguments='{"n":5}', name='factorial')]
---------- ToolCallExecutionEvent (factorial_creator) ----------
[FunctionExecutionResult(content='120', name='factorial', call_id='call_wpLjDwnZ3zRQFu1Kaw35msnN', is_error=False)]
---------- ModelClientStreamingChunkEvent (factorial_creator) ----------
The factorial of 5, written as 5!, is 120.


### Conceptually, agent is like a person with
1. System message : Character of the person
2. Tools : what all it can do apart from speaking about the vast info available through the model
3. Model : its heart / brain